<img src="https://comet.ml/images/logo_comet_light.png" width="200px"/>

# Comet.ml REST API

Comet.ml has an extensive interface to all of your data using a [REST API](https://en.wikipedia.org/wiki/Representational_state_transfer) through [Comet.ml endpoints](https://www.comet.ml/docs/rest-api/endpoints/). Now, you can access this information easily through the Comet.ml Python SDK. Requires version comet_ml version 1.0.40 or greater.

## Setup

To run the following experiments, you'll need to set your COMET_API_KEY and COMET_REST_API_KEY. The easiest way to to this is to set the values in a cell like this:

```python
%%writefile .env
COMET_API_KEY="..."
COMET_REST_API_KEY=".."
```
(where you replace the ...'s with your keys.

You can get your COMET_API_KEY under your quickstart link (replace YOURID with your Comet.ml ID):

https://www.comet.ml/YOURID/quickstart

You can get your COMET_REST_API_KEY under your settings (replace YOURID with your Comet.ml ID):

https://www.comet.ml/YOURID/settings/account

## Quick Overview

To access the REST API through the comet.ml SDK, you will need to make an API() instance. First, we import the API class:

In [1]:
from comet_ml import API

and creat the instance:

In [2]:
api = API()

Using the `api` instance, you can get the name of your workspaces:

In [3]:
api.workspaces

['dsblank']

If you reference your workspace by name from api.workspaces, you'll see your projects:

In [4]:
api.workspaces["dsblank"]

['pytorch', 'mnist-001', 'visualizations', 'testing', 'pypy', 'tensorflow', 'chainer', 'mnist-014', 'optimizer', 'general', 'jupyter-experiments', 'keras', 'fastai']

Or, get the projects from another user's workspace:

In [5]:
api.workspaces["cometpublic"]

['fasttext', 'comet-notebooks', 'parameter-space-exploration', 'home-credit']

Using the same method, you can refer to a project by name:

In [6]:
api.workspaces["cometpublic"]["comet-notebooks"]

['7092a5e4c362453fb0b3f06785a1d30c', 'd21f94a1c71841d2961da1e6ddb5ab20']

Or, using the slash delimiter:

In [7]:
api.workspaces["cometpublic/comet-notebooks"]

['7092a5e4c362453fb0b3f06785a1d30c', 'd21f94a1c71841d2961da1e6ddb5ab20']

And one more level, get the details of an experiment:

In [8]:
api.workspaces["cometpublic"]["comet-notebooks"]['d21f94a1c71841d2961da1e6ddb5ab20']

<Experiment 'cometpublic/comet-notebooks/d21f94a1c71841d2961da1e6ddb5ab20'>

Or, again using the slash shorthand:

In [9]:
api.workspaces["cometpublic/comet-notebooks/d21f94a1c71841d2961da1e6ddb5ab20"]

<Experiment 'cometpublic/comet-notebooks/d21f94a1c71841d2961da1e6ddb5ab20'>

Let's get the experiment and save it to a variable named `exp`:

In [10]:
exp = api.workspaces["cometpublic"]["comet-notebooks"]['d21f94a1c71841d2961da1e6ddb5ab20']

In [11]:
exp

<Experiment 'cometpublic/comet-notebooks/d21f94a1c71841d2961da1e6ddb5ab20'>

There are a number of items you get from the Experiment:

In [12]:
help(exp)

Help on Experiment in module comet_ml.api object:

class Experiment(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, api, workspace, project_data, experiment_key, name=None)
 |      API Experiment interface.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  display(self, clear=False, wait=True, new=0, autoraise=True)
 |      Show the comet.ml experiment page in an IFrame in a
 |      Jupyter notebook or Jupyter lab, OR open a browser
 |      window or tab.
 |      
 |      For Jupyter environments:
 |      
 |      Args:
 |          clear: to clear the output area, use clear=True
 |          wait: to wait for the next displayed item, use
 |                wait=True (cuts down on flashing)
 |      
 |      For non-Jupyter environments:
 |      
 |      Args:
 |          new: open a new browser window if new=1, otherwise re-use
 |               existing window/tab
 |          autoraise: make the browser tab/window active
 |  
 |  ------------------------------

For example, we can explore the `other` property, which shows items saved with Experiment.log_other(NAME, VALUE):

In [13]:
exp.other

[{'name': 'Name',
  'valueMax': 'example 001',
  'valueMin': 'example 001',
  'valueCurrent': 'example 001',
  'timestampMax': 1543866627717,
  'timestampMin': 1543866627717,
  'timestampCurrent': 1543866627717}]

In this example, we see that the experiment has the `Name` "example 001". We can use `Name` to also look up experiments:

In [14]:
api.workspaces["cometpublic/comet-notebooks/example 001"]

<Experiment 'cometpublic/comet-notebooks/example 001'>

That's it for a quick overview. Now let's look in detail at each component, and introduce the low-level REST API as well.

## Workspaces

By default, api.workspaces reports only your workspace names:

In [15]:
api.workspaces

['dsblank']

You can also interate over those names:

In [16]:
for workspace in api.workspaces:
    print(workspace)

dsblank


As we saw above, you can also access other public workspaces as well:

In [17]:
api.workspaces["cometpublic"]

['fasttext', 'comet-notebooks', 'parameter-space-exploration', 'home-credit']

## Projects

Under the workspace, you'll find the projects:

In [18]:
api.workspaces["cometpublic"]

['fasttext', 'comet-notebooks', 'parameter-space-exploration', 'home-credit']

In [19]:
project = api.workspaces["cometpublic"]["comet-notebooks"]
## OR:
#project = api.workspaces["cometpublic/comet-notebooks"]

If you just print out, or iterate over a project, you get access to the experiment ids:

In [20]:
project

['7092a5e4c362453fb0b3f06785a1d30c', 'd21f94a1c71841d2961da1e6ddb5ab20']

However, you can also access all of the project data via the `.data` property:

In [21]:
project.data

{'project_id': '63776506878949eb9ac225a2a24b87f6',
 'user_name': 'cometpublic',
 'project_name': 'comet-notebooks',
 'project_desc': '',
 'team_id': 'cometpublic-default',
 'is_owner': False,
 'is_public': True,
 'is_shared': False,
 'num_of_experiments': 2,
 'last_updated': 1543867086997,
 'team_name': 'cometpublic',
 'experiments': ['7092a5e4c362453fb0b3f06785a1d30c',
  'd21f94a1c71841d2961da1e6ddb5ab20']}

And that's everything there is to a project object:

In [22]:
help(project)

Help on Experiments in module comet_ml.api object:

class Experiments(builtins.object)
 |  Methods defined here:
 |  
 |  __getitem__(self, item)
 |  
 |  __init__(self, api, workspace, project_data)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |  
 |  __len__(self)
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  data
 |      The project data in JSON format.



## Experiments

Continuing with the dictionary-like access, you can see and iterate over the experiment ids:

In [23]:
api.workspaces["cometpublic"]["comet-notebooks"]

['7092a5e4c362453fb0b3f06785a1d30c', 'd21f94a1c71841d2961da1e6ddb5ab20']

In [24]:
exp = api.workspaces["cometpublic"]["comet-notebooks"]['d21f94a1c71841d2961da1e6ddb5ab20']
## OR
# exp = api.workspaces["cometpublic/comet-notebooks/d21f94a1c71841d2961da1e6ddb5ab20"]

In addition, experiments also have a `.data` property:

In [25]:
exp.data

{'code_sha': '66e8551f',
 'file_name': 'Jupyter interactive',
 'file_path': 'Jupyter interactive',
 'duration_millis': 1043394000,
 'start_server_timestamp': 1542823692825,
 'end_server_timestamp': 1543867086997,
 'has_images': False,
 'experiment_key': 'd21f94a1c71841d2961da1e6ddb5ab20',
 'metrics': ['train_accuracy',
  'train_curr_epoch',
  'train_loss',
  'train_val_loss'],
 'other': ['Name'],
 'parameters': ['f']}

In this brief dictionary representation, you will see that `other`, `metrics` and `parameters` give a list of names. However, as we saw above, you can get more information through properties of the same names:

In [26]:
exp.other

[{'name': 'Name',
  'valueMax': 'example 001',
  'valueMin': 'example 001',
  'valueCurrent': 'example 001',
  'timestampMax': 1543866627717,
  'timestampMin': 1543866627717,
  'timestampCurrent': 1543866627717}]

In [27]:
exp.other[0]["name"], exp.other[0]["valueCurrent"]

('Name', 'example 001')

In [28]:
exp.metrics

[{'name': 'train_accuracy',
  'valueMax': '0.0',
  'valueMin': '0.0',
  'valueCurrent': '0.0',
  'timestampMax': 1542823699380,
  'timestampMin': 1542823699380,
  'timestampCurrent': 1542823704609,
  'runContextMax': 'train',
  'runContextMin': 'train',
  'runContextCurrent': 'train',
  'stepMax': 1,
  'stepMin': 1,
  'stepCurrent': 3},
 {'name': 'train_curr_epoch',
  'valueMax': '1.0',
  'valueMin': '0.0',
  'valueCurrent': '1.0',
  'timestampMax': 1542823693885,
  'timestampMin': 1542823693885,
  'timestampCurrent': 1542823699380,
  'runContextMax': 'train',
  'runContextMin': 'train',
  'runContextCurrent': 'train'},
 {'name': 'train_loss',
  'valueMax': '2.3872790336608887',
  'valueMin': '2.3872790336608887',
  'valueCurrent': '2.3872790336608887',
  'timestampMax': 1542823695820,
  'timestampMin': 1542823695820,
  'timestampCurrent': 1542823695820,
  'runContextMax': 'train',
  'runContextMin': 'train',
  'runContextCurrent': 'train',
  'stepMax': 0,
  'stepMin': 0,
  'stepCurren

In [29]:
exp.parameters

[{'name': 'f',
  'valueMax': '/run/user/1000/jupyter/kernel-b1c4403c-c673-4e7f-90dc-853616d231e2.json',
  'valueMin': '/run/user/1000/jupyter/kernel-b1c4403c-c673-4e7f-90dc-853616d231e2.json',
  'valueCurrent': '/run/user/1000/jupyter/kernel-b1c4403c-c673-4e7f-90dc-853616d231e2.json',
  'timestampMax': 1542823692798,
  'timestampMin': 1542823692798,
  'timestampCurrent': 1542823692798}]

You can see all of the methods and propeties on an experiment instance:

In [30]:
help(exp)

Help on Experiment in module comet_ml.api object:

class Experiment(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, api, workspace, project_data, experiment_key, name=None)
 |      API Experiment interface.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  display(self, clear=False, wait=True, new=0, autoraise=True)
 |      Show the comet.ml experiment page in an IFrame in a
 |      Jupyter notebook or Jupyter lab, OR open a browser
 |      window or tab.
 |      
 |      For Jupyter environments:
 |      
 |      Args:
 |          clear: to clear the output area, use clear=True
 |          wait: to wait for the next displayed item, use
 |                wait=True (cuts down on flashing)
 |      
 |      For non-Jupyter environments:
 |      
 |      Args:
 |          new: open a new browser window if new=1, otherwise re-use
 |               existing window/tab
 |          autoraise: make the browser tab/window active
 |  
 |  ------------------------------

For example, just like when creating and logging data, you can also use the `.display()` method to show the Comet.ml page for that experiment right in the notebook:

In [31]:
exp.display()

You can get an existing experiment:

In [33]:
e = api.workspaces["dsblank"]["chainer"]["596d91ae1dbc420c9b13a3ced858de3c"]
ee = e.existing_experiment

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/dsblank/chainer/596d91ae1dbc420c9b13a3ced858de3c



You can make changes to the saved data using the existing experiment:

https://www.comet.ml/docs/python-sdk/ExistingExperiment/

In [34]:
ee.end()

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/dsblank/chainer/596d91ae1dbc420c9b13a3ced858de3c



### Examples

Comet.ml is working on a query API which will allow highly effcient queries of your data. However, you can also write your own query of sorts.

Here is some code that prints out the names of experiments that have associated HTML (this can take a long time if you have many experiments):

In [32]:
for workspace in api.workspaces:
    for project in api.workspaces[workspace]:
        for exp_id in api.workspaces[workspace][project]:
            exp = api.workspaces[workspace][project][exp_id]
            if exp.html != None:
                print(exp)

<Experiment 'dsblank/chainer/596d91ae1dbc420c9b13a3ced858de3c'>
<Experiment 'dsblank/chainer/698e577fc880463381e179a2125c679a'>
<Experiment 'dsblank/chainer/f9fbef4c629848b495050229295d8d9d'>
<Experiment 'dsblank/chainer/5b8a2e2d89b44c3fa1e9eb2c6b4d9763'>
<Experiment 'dsblank/chainer/dd760f7cafb54bd591fb02ee79e3b328'>
<Experiment 'dsblank/chainer/81b743e020924c399321e3c0a8a8638e'>
<Experiment 'dsblank/chainer/87fcbef35def4dc8986cc761c6344486'>
<Experiment 'dsblank/chainer/0a8700299154434898228a47f93f18bf'>
<Experiment 'dsblank/chainer/0d1e22d1572f431682904058313e32ef'>
<Experiment 'dsblank/chainer/42c666e3632a4975aa09ee955e3813de'>
<Experiment 'dsblank/chainer/8bc73ecb2fa943adb8cc949d1888d06d'>
<Experiment 'dsblank/chainer/3e7adc06a2584a97b3d58cf5328ac06b'>
<Experiment 'dsblank/chainer/edfa1479ae8347a2bd491cfdca1dcd10'>
<Experiment 'dsblank/chainer/80ac2ab19af3428888ab63576fe23d16'>


Here is a function that will find the first experiment that has associated images:

In [6]:
def find_image():
    for workspace in api.workspaces:
        for project in api.workspaces[workspace]:
            for exp_id in api.workspaces[workspace][project]:
                exp = api.workspaces[workspace][project][exp_id]
                if exp.images != []:
                    return exp

In [7]:
find_image()

<Experiment 'dsblank/pytorch/3b56611892b7447aa8c4486a6eeb27d0'>

Now, we get the experiment API, and explore the `.images` property:

In [35]:
api.workspaces['dsblank/pytorch/3b56611892b7447aa8c4486a6eeb27d0'].images

[{'apiKey': 'x0yluTXX4ysG4z1k0FaTAz3qj',
  'runId': 'EgBEDhAJMA6OxWXZ1dHv4AkSz',
  'experimentKey': '3b56611892b7447aa8c4486a6eeb27d0',
  'projectId': '1f1999d917104dbe8b6a17b2c28c171c',
  'figCounter': 0,
  'figName': 'Exponential Slowdown',
  'step': None,
  'runContext': None,
  'fileName': '4ykmnOHJFZkrRD36KDrTXvHJk.svg',
  'imagePath': 'https://s3.amazonaws.com/comet.ml/4ykmnOHJFZkrRD36KDrTXvHJk.svg'}]

We can get a URL for the image, and display it in the notebook:

In [36]:
url = api.workspaces['dsblank/pytorch/3b56611892b7447aa8c4486a6eeb27d0'].images[0]["imagePath"]
url

'https://s3.amazonaws.com/comet.ml/4ykmnOHJFZkrRD36KDrTXvHJk.svg'

In [37]:
from IPython.display import Image

In [38]:
Image(url=url)

Now, let's write a short program that will find the run with the best accuracy given a workspace/project string:

In [53]:
def find_best_run(project):
    runs = []
    for exp_id in api.workspaces[project]:
        exp = api.workspaces[project][exp_id]
        accs = [x["valueMax"] for x in exp.metrics if x["name"] == "acc"]
        if len(accs) > 0:
            runs.append([float(accs[0]), exp])
    if runs:
        return sorted(runs, key=lambda v: v[0], reverse=True)[0]

In [54]:
find_best_run("cometpublic/fasttext")

[1.0, <Experiment 'cometpublic/fasttext/44ea2d68794d4c5e9e18b86c30562bf8'>]

## Low-level API

If you would prefer a more direct, lower-level API, you can also use the following functions from the API() instance. These correspond to the details in:

https://development.comet.ml/docs/rest-api/endpoints/

### Utility functions

*    get_url(version=None)
*    get_url_server(version=None)
*    get_url_end_point(end_point, version=None)
*    get_request(end_point, params)
*    get_version(self)

### Access functions
*    get_workspaces(workspace_name=None)
*    get_projects(workspace)
*    get_experiment_keys(project_id)
*    get_experiment_data(project_id)
*    get_experiment(experiment_key)
*    get_experiment_html(experiment_key)
*    get_experiment_code(experiment_key)
*    get_experiment_stdout(experiment_key)
*    get_experiment_installed_packages(experiment_key)
*    get_experiment_graph(experiment_key)
*    get_experiment_images(experiment_key)
*    get_experiment_parameters(experiment_key, param=None)
*    get_experiment_metrics(experiment_key, metric=None)
*    get_experiment_other(experiment_key, other=None, value=None)
*    get_experiment_metrics_raw(experiment_key, metric=None)

We hope that this gives you some ideas of how you can use the REST API!